# Medicane Zorbas (september 2018) : ECMWF analysis and deterministic forecast

*Author : Frédéric FERRY - Météo-France / Ecole Nationale de la Météorologie (June 2023)*

Python concepts :
- Open netcdf files (xarray)
- Compute equivalent potential temperature (metpy)
- Create maps and animations (matplotlib, cartopy)

In [ ]:
import os

import xarray as xr
import numpy as np
import pandas as pd

from shapely.geometry import Point

import metpy.calc as mpcalc
from metpy.units import units

from scipy.ndimage import gaussian_filter
from scipy.ndimage import maximum_filter, minimum_filter

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob
from tqdm import tqdm

In [ ]:
def make_animation(gif_filepath):
    from PIL import Image
    import os
    from IPython.display import Image as IPImage
    from IPython.display import display
    import time
    
    image_folder = './anim/' # répertoire contenant les fichiers PNG
    output_file = gif_filepath # nom du fichier de sortie
    animation_speed = 0.9 # vitesse de l'animation en secondes
    
    # Liste tous les fichiers PNG dans le répertoire image_folder
    files = sorted(os.listdir(image_folder))
    image_files = [f for f in files if f.endswith('.png')]
    
    # Ouvre chaque fichier PNG et ajoute l'image à une liste
    images = []
    for filename in image_files:
        img = Image.open(os.path.join(image_folder, filename))
        images.append(img)
    
    # Crée l'animation GIF
    images[0].save(output_file, save_all=True, append_images=images[1:], duration=int(animation_speed*1000), loop=0)
    # Affiche l'animation GIF dans Jupyter
    with open(output_file,'rb') as f:
        display(IPImage(data=f.read(), format='png'))
    # Efface les fichiers PNG
    for filename in image_files:
        os.remove(image_folder+filename)

In [ ]:
projection=ccrs.PlateCarree()

def plot_background(ax):
    ax.coastlines()
    ax.gridlines()
    ax.set_xticks(np.linspace(-180, 180, 37), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(-90, 90, 37), crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    return(ax)

In [ ]:
def plot_maxmin_points(data, extrema, nsize, symbol, color='k',
                       plotValue=True, transform=None):

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        ax.text(data.longitude[mxx[i]].values, data.latitude[mxy[i]].values, symbol, color=color, size=12,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax.text(data.longitude[mxx[i]].values, data.latitude[mxy[i]].values,
                '\n' + str(int(data[mxy[i], mxx[i]])),
                color=color, size=10, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

def print_maxmin_points(data, extrema, nsize):
    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        # Print date lon lat and pressure of the minimum
        print(date, data.longitude[mxx[i]].values, data.latitude[mxy[i]].values, int(data[mxy[i], mxx[i]]))

# Part 1 : ECMWF analysis (2018-09-24T00 to 2018-09-30T12)

In [ ]:
Ana_Z500 = './data/Z500_September2018_HiRes.nc'
Ana_T500 = './data/T500_September2018_HiRes.nc'
Ana_MSLP = './data/MSLP_September2018_HiRes.nc'
Ana_PV320 = './data/PV320_September2018_HiRes.nc'
Ana_qT850 = './data/qT850_September2018_HiRes.nc'

data_z    = xr.open_dataset(Ana_Z500)
data_t500    = xr.open_dataset(Ana_T500)
data_mslp   = xr.open_dataset(Ana_MSLP)
data_pv   = xr.open_dataset(Ana_MSLP)
data_qT   = xr.open_dataset(Ana_qT850)

print(data_z)
print(data_z.time.values)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>What is the temporal resolution of the available ECMWF analysis data ?</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

In [ ]:
t0 ='2018-09-24T00'
t1 ='2018-09-30T12'

data_z    = xr.open_dataset(Ana_Z500).sel(time=slice(t0,t1))
data_t500    = xr.open_dataset(Ana_T500).sel(time=slice(t0,t1))
data_mslp   = xr.open_dataset(Ana_MSLP).sel(time=slice(t0,t1))
data_pv   = xr.open_dataset(Ana_PV320).sel(time=slice(t0,t1))
data_qT   = xr.open_dataset(Ana_qT850).sel(time=slice(t0,t1))

print(data_z)

In [ ]:
z_ana = data_z['z']/9.81
t500_ana = data_t500['t']-273.15
mslp_ana = data_mslp['msl']/100
pv_ana = data_pv['pv']*1e6
q_ana = data_qT['q']
T_ana = data_qT['t']

lats = z_ana.latitude.values
lons = z_ana.longitude.values

time = z_ana.time.values
times_str=[x for x in range(len(time))]
dates_str=[x for x in range(len(time))]
for i in range(len(time)):
	times_str[i] = str(time[i])
	dates_str[i] = times_str[i][0:13]
print(dates_str)

In [ ]:
Td = mpcalc.dewpoint_from_specific_humidity(850* units.hPa, T_ana.values* units.kelvin, q_ana.values* units('kg/kg'))
Thetae_ana=mpcalc.equivalent_potential_temperature(850* units.hPa, T_ana, Td)
print(Thetae_ana.shape)
print(np.min(Thetae_ana))
print(np.max(Thetae_ana))

In [ ]:
cmap='jet'
clevs1 = np.linspace(5500, 6000, 21)
clevs2 = np.arange(994, 1040, 2)
clevs3 = np.linspace(1, 12, 21)
clevs4 = np.arange(270,345,5)
clevs5=np.arange(-30,2.5,2.5)

for i in tqdm(range(len(time))):
    
    #print(dates_str[i])

    fig = plt.figure(figsize=(10, 5))
    fig.suptitle('Analysis : '+dates_str[i], fontsize=14)
    
    ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
    plt.title('MSLP and $\Theta_E$ at 850 hPa', fontsize=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, Thetae_ana[i,:,:], levels=clevs4, cmap=cmap, extend='both',transform=ccrs.PlateCarree())
    c = ax.contour(lons, lats, mslp_ana[i,:,:], levels=clevs2, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
    cb.set_label('K', fontsize=12)
    
    plt.close()
    figname='./anim/MSLP_Thetae850_'+dates_str[i]
    fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
gif_filepath = './anim/MSLP_Thetae850_an.gif'
make_animation(gif_filepath)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Give an approximate equivalent potential temperature range associated to medicane Zorbas.</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

In [ ]:
for i in tqdm(range(len(time))):
    
    #print(dates_str[i])
    
    fig = plt.figure(figsize=(10, 5))
    fig.suptitle('Analysis : '+dates_str[i], fontsize=14)

    ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
    plt.title('Geopotential height and temperature at 500 hPa', size=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, t500_ana[i,:,:], levels=clevs5, cmap=cmap, extend='both',transform=ccrs.PlateCarree())
    c = ax.contour(lons, lats, z_ana[i,:,:], levels=clevs1, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
    cb.set_label('°C', fontsize=12)
    
    plt.close()
    figname='./anim/ZT500_'+dates_str[i]
    fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
gif_filepath = './anim/ZT500_an.gif'
make_animation(gif_filepath)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>What upper level structure approaches Medicane Zorbas just before its initiation ?</p>
<p><b>2) </b>At a larger scale, what type of Rossby wave breaking (cyclonic/anticyclonic) is linked to this feature ?</p>
    
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

In [ ]:
for i in tqdm(range(len(time))):
    
    #print(dates_str[i])

    fig = plt.figure(figsize=(10, 5))
    fig.suptitle('Analysis : '+dates_str[i], fontsize=14)
    
    ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
    plt.title('Potential vorticity at 320K', fontsize=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, pv_ana[i,:,:], levels=clevs3, cmap=cmap, extend='max',transform=ccrs.PlateCarree())
    #c = ax.contour(lons, lats, pv_ana[i,:,:], levels=clevs3, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
    #ax.clabel(c, fmt='%4.1i', fontsize=10)
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
    cb.set_label('PVU', fontsize=12)
    
    plt.close()
    figname='./anim/PV320_'+dates_str[i]
    fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
gif_filepath = './anim/PV320_an.gif'
make_animation(gif_filepath)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>What are the 320K (i.e around 300hPa) potential vorticity values associated to the upper level structure described above ? What can we deduce concerning the origin of the corresponding air (tropospheric/stratospheric) ? </p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

<div class="alert alert-danger">
<p><b>Reduce the geographical domain to 25N-50N 0W-30E.</b></p>
</div>

In [ ]:
latS = 
latN = 
lonW = 
lonE = 

In [ ]:
z_ana = z_ana.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
t500_ana = t500_ana.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
mslp_ana = mslp_ana.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
pv_ana = pv_ana.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
Thetae_ana = Thetae_ana.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))

lats = z_ana.latitude.values
lons = z_ana.longitude.values

In [ ]:
cmap='jet'
clevs1 = np.linspace(5500, 6000, 21)
clevs2 = np.arange(994, 1040, 2)
clevs3 = np.linspace(1, 12, 21)
clevs4 = np.arange(270,345,5)
clevs5=np.arange(-30,2.5,2.5)

In [ ]:
for i in tqdm(range(len(time))):
    
    #print(dates_str[i])

    fig = plt.figure(figsize=(15, 8))
    fig.suptitle('Analysis : '+dates_str[i], fontsize=14)

    ax = fig.add_subplot(121, projection=ccrs.PlateCarree())
    plt.title('Geopotential height and temperature at 500 hPa', size=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, t500_ana[i,:,:], levels=clevs5, cmap=cmap, extend='both',transform=ccrs.PlateCarree())
    c = ax.contour(lons, lats, z_ana[i,:,:], levels=clevs1, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
    cb.set_label('°C', fontsize=12)
    
    ax = fig.add_subplot(122, projection=ccrs.PlateCarree())
    plt.title('MSLP and $\Theta_E$ at 850 hPa', fontsize=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, Thetae_ana[i,:,:], levels=clevs4, cmap=cmap, extend='both',transform=ccrs.PlateCarree())
    c = ax.contour(lons, lats, mslp_ana[i,:,:], levels=clevs2, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)
    #plot_maxmin_points(lon_grid, lat_grid, mslp_ana[i,:,:], 'max', 25,
    #symbol='H', color='r',  transform=ccrs.PlateCarree())
    plot_maxmin_points(mslp_ana[i,:,:], 'min', 25,
                       symbol='L', color='b', transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
    cb.set_label('K', fontsize=12)
    
    #plt.show()
    plt.close()
    figname='./anim/MSLP_Thetae850_ZT500'+dates_str[i]
    fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
gif_filepath = './anim/MSLP_Thetae850_ZT500_an2.gif'
make_animation(gif_filepath)

In [ ]:
for i in tqdm(range(len(time))):
    
    #print(dates_str[i])

    fig = plt.figure(figsize=(15, 8))
    fig.suptitle('Analysis : '+dates_str[i], fontsize=14)
    
    ax = fig.add_subplot(121, projection=ccrs.PlateCarree())
    plt.title('Potential vorticity at 320K', fontsize=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, pv_ana[i,:,:], levels=clevs3, cmap=cmap, extend='max',transform=ccrs.PlateCarree())
    #c = ax.contour(lons, lats, pv_ana[i,:,:], levels=clevs3, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
    #ax.clabel(c, fmt='%4.1i', fontsize=10)
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
    cb.set_label('PVU', fontsize=12)

    ax = fig.add_subplot(122, projection=ccrs.PlateCarree())
    plt.title('MSLP and $\Theta_E$ at 850 hPa', fontsize=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, Thetae_ana[i,:,:], levels=clevs4, cmap=cmap, extend='both',transform=ccrs.PlateCarree())
    c = ax.contour(lons, lats, mslp_ana[i,:,:], levels=clevs2, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)
    #plot_maxmin_points(lon_grid, lat_grid, mslp_ana[i,:,:], 'max', 25,
    #symbol='H', color='r',  transform=ccrs.PlateCarree())
    plot_maxmin_points(mslp_ana[i,:,:], 'min', 25,
                       symbol='L', color='b', transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
    cb.set_label('K', fontsize=12)
    
    plt.close()
    figname='./anim/MSLP_Thetae850_PV320'+dates_str[i]
    fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
gif_filepath = './anim/MSLP_Thetae850_PV320_an2.gif'
make_animation(gif_filepath)

# Part 2 : ECMWF deterministic forecast

In [ ]:
Fcst_Z = './data/Z500_fc_20180924.nc'
Fcst_T500 = './data/T500_fc_20180924.nc'
Fcst_MSLP = './data/MSLP_fc_20180924.nc'
Fcst_PV320 = './data/PV320_fc_20180924.nc'
Fcst_qT850 = './data/qT850_fc_20180924.nc'

data_zf    = xr.open_dataset(Fcst_Z)
data_t500f    = xr.open_dataset(Fcst_T500)
data_mslpf    = xr.open_dataset(Fcst_MSLP)
data_pvf    = xr.open_dataset(Fcst_PV320)
data_qTf    = xr.open_dataset(Fcst_qT850)

print(data_zf)
print(data_zf.time.values)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>What is the temporal resolution of the ECMWF forecast data ? Is it regular ?</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

<div class="alert alert-danger">
<b>To have the same spatio-temporal resolution as the zoomed analysis, specify a regular 6-hourly time slice from 2018-09-24T00 to 2018-09-30T12 and reduce the geographical domain to 25N-50N 0W-30E. </b>
    
<p><b>Hint : the pandas date_range function may be useful :
    
https://pandas.pydata.org/docs/reference/api/pandas.date_range.html.</b></p>
</div>

In [ ]:
t0 ='2018-09-24T00'
t1 ='2018-09-30T12'
dates = 

latS = 
latN = 
lonW = 
lonE = 

In [ ]:
data_zf    = xr.open_dataset(Fcst_Z).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE)).sel(time=dates)
data_t500f    = xr.open_dataset(Fcst_T500).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE)).sel(time=dates)
data_mslpf    = xr.open_dataset(Fcst_MSLP).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE)).sel(time=dates)
data_pvf    = xr.open_dataset(Fcst_PV320).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE)).sel(time=dates)
data_qT850f    = xr.open_dataset(Fcst_qT850).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE)).sel(time=dates)

print(data_zf)

In [ ]:
z_fcst=data_zf['z']/9.81
t500_fcst=data_t500f['t']-273.15
mslp_fcst=data_mslpf['msl']/100
pv_fcst=data_pvf['pv']*1e6
q850_fcst=data_qT850f['q']
t850_fcst=data_qT850f['t']

lats = z_fcst.latitude.values
lons = z_fcst.longitude.values

time = z_fcst.time.values
times_str=[x for x in range(len(time))]
dates_str=[x for x in range(len(time))]
for i in range(len(time)):
	times_str[i] = str(time[i])
	dates_str[i] = times_str[i][0:13]
print(dates_str)

In [ ]:
Td = mpcalc.dewpoint_from_specific_humidity(850* units.hPa, t850_fcst.values* units.kelvin, q850_fcst.values* units('kg/kg'))
Thetae_fcst=mpcalc.equivalent_potential_temperature(850* units.hPa, t850_fcst, Td)
print(Thetae_fcst.shape)
print(np.min(Thetae_fcst))
print(np.max(Thetae_fcst))

In [ ]:
for i in tqdm(range(len(time))):
    
    #print(dates_str[i])

    fig = plt.figure(figsize=(15, 8))
    fig.suptitle('Analysis : '+dates_str[0]+' - Forecast : '+dates_str[i], fontsize=14)

    ax = fig.add_subplot(121, projection=ccrs.PlateCarree())
    plt.title('Geopotential height and temperature at 500 hPa', size=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, t500_fcst[i,:,:], levels=clevs5, cmap=cmap, extend='both',transform=ccrs.PlateCarree())
    c = ax.contour(lons, lats, z_fcst[i,:,:], levels=clevs1, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
    cb.set_label('°C', fontsize=12)
    
    ax = fig.add_subplot(122, projection=ccrs.PlateCarree())
    plt.title('MSLP and $\Theta_E$ at 850 hPa', fontsize=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, Thetae_fcst[i,:,:], levels=clevs4, cmap=cmap, extend='both',transform=ccrs.PlateCarree())
    c = ax.contour(lons, lats, mslp_fcst[i,:,:], levels=clevs2, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)
    #plot_maxmin_points(lon_grid, lat_grid, mslp_ana[i,:,:], 'max', 25,
    #symbol='H', color='r',  transform=ccrs.PlateCarree())
    plot_maxmin_points(mslp_fcst[i,:,:], 'min', 25,
                       symbol='L', color='b', transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
    cb.set_label('K', fontsize=12)
    
    #plt.show()
    plt.close()
    figname='./anim/MSLP_Thetae850_ZT500_'+dates_str[i]
    fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
gif_filepath = './anim/MSLP_Thetae850_ZT500_fc.gif'
make_animation(gif_filepath)

In [ ]:
for i in tqdm(range(len(time))):
    
    #print(dates_str[i])

    fig = plt.figure(figsize=(15, 8))
    fig.suptitle('Analysis : '+dates_str[0]+' - Forecast : '+dates_str[i], fontsize=14)
    
    ax = fig.add_subplot(121, projection=ccrs.PlateCarree())
    plt.title('Potential vorticity at 320K', fontsize=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, pv_fcst[i,:,:], levels=clevs3, cmap=cmap, extend='max',transform=ccrs.PlateCarree())
    #c = ax.contour(lons, lats, pv_fcst[i,:,:], levels=clevs3, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
    #ax.clabel(c, fmt='%4.1i', fontsize=10)
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
    cb.set_label('PVU', fontsize=12)

    ax = fig.add_subplot(122, projection=ccrs.PlateCarree())
    plt.title('MSLP and $\Theta_E$ at 850 hPa', fontsize=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, Thetae_fcst[i,:,:], levels=clevs4, cmap=cmap, extend='both',transform=ccrs.PlateCarree())
    c = ax.contour(lons, lats, mslp_fcst[i,:,:], levels=clevs2, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)
    #plot_maxmin_points(lon_grid, lat_grid, mslp_ana[i,:,:], 'max', 25,
    #symbol='H', color='r',  transform=ccrs.PlateCarree())
    plot_maxmin_points(mslp_fcst[i,:,:], 'min', 25,
                       symbol='L', color='b', transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
    cb.set_label('K', fontsize=12)
    
    plt.close()
    figname='./anim/MSLP_Thetae850_PV320_'+dates_str[i]
    fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
gif_filepath = './anim/MSLP_Thetae850_PV320_fc.gif'
make_animation(gif_filepath)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Does the september 24th 00UTC run propose a medicane-like scenario for September 27th-28th ?</p>
<p><b>2) </b>Do you see differences in the initiation, track and intensity of the system between the deterministic forecast and the analysis ?</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

# Part 3 : differences between analysis and deterministic forecast

<div class="alert alert-danger">
<b>Compute differences between the analysis and the forecast (arrays z_diff and mslp_diff) </b>
</div>

In [ ]:
z_diff=
mslp_diff=
print(z_diff.shape)

In [ ]:
cmap='coolwarm'
clevsa1 = np.arange(-50, 55, 5)
clevsa2 = np.arange(-8, 9, 1)

for i in tqdm(range(len(time)-1)):
    #print(dates_str[i])
    fig = plt.figure(figsize=(15, 8))
    fig.suptitle('Analysis minus Forecast : '+dates_str[i], fontsize=14)
    ax = fig.add_subplot(121, projection=ccrs.PlateCarree())
    plt.title('Geopotential height at 500 hPa', size=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, z_diff[i,:,:], clevsa1, cmap=cmap, extend='both', transform=ccrs.PlateCarree())
    c = ax.contour(lons, lats, z_diff[i,:,:], clevsa1, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
    cb.set_label('(m)', fontsize=12)
    
    ax = fig.add_subplot(122, projection=ccrs.PlateCarree())
    plt.title('Mean Sea Level Pressure', fontsize=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, mslp_diff[i,:,:], clevsa2, cmap=cmap, extend='both', transform=ccrs.PlateCarree())
    c = ax.contour(lons, lats, mslp_diff[i,:,:], clevsa2, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
    cb.set_label('(hPa)', fontsize=12)
    
    plt.close()
    figname='./anim/Z500_MSLP_diff_'+dates_str[i]
    fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
gif_filepath = './anim/Z500_MSLP_an_minus_fc.gif'
make_animation(gif_filepath)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>When do the differences between the analysis and the deterministic forecast start to be significant ?</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

# Extra task : tracking Zorbas in the analysis and deterministic forecasts

<div class="alert alert-danger">

<p><b>1) Use the code from the previous notebooks to track the medicane Zorbas in the analysis and deterministic forecast from 2018-09-27T12 to 2018-09-30T12.</b></p>

<p><b>2) Highlight the differences between the analysis and the deterministic forecast by ploting maps of the tracks and curves of the evolution of the MSLP.</b></p>


</div>